In [1]:
import pandas as pd

In [2]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

In [3]:
#Leo el events con mis datos numericos

events_df = pd.read_csv("/home/miki_mustard/Desktop/Facultad/Datos/TP2/events_con_otros_supports.csv", low_memory=False)

In [ ]:
events_df.columns

In [ ]:
#Intento de mean encoding para new_vs_returning
events_df.new_vs_returning.value_counts(normalize=True)

In [8]:
events_df.groupby(['person'])

In [ ]:
def mean_encoding(columna, label):
    encodeado = list()
    for i in range(0, len(columna)):
        if (label[i] is 0) and (columna[i] is 'New'):
            encodeado.append(0.949022)
        elif (label[i] is 0) and (columna[i] is 'Returning'):
            encodeado.append(0.904392)
        elif (label[i] is 1) and (columna[i] is 'New'): 
            encodeado.append(0.050978)
        else:
            encodeado.append(0.095608)
    return encodeado    

In [ ]:
encodeado_n_vs_r = mean_encoding(train_df.new_vs_returning, train_df.label)

In [ ]:
events_df.shape

In [ ]:
df = pd.DataFrame()
df['enc_n_vs_r'] = encodeado_n_vs_r
df['person'] = train_df.person

In [9]:
#Tiro las columnas que no me interesan 
events_df = events_df.drop(columns=['event', 'skus', 'new_vs_returning', 'region', 'device_type', 'screen_resolution'
                                   ,'samsung', 'motorola', 'iphone', 'asus', 'lg', 'sony'])

In [ ]:
events_df.columns

In [10]:
from sklearn import preprocessing

In [11]:
le = preprocessing.LabelEncoder()

In [12]:
le.fit(events_df.person)

LabelEncoder()

In [ ]:
def generar_id(personas):
    dict_personas = {}
    i = 0
    for persona in personas:
        dict_personas[persona] = i
        i = i+1
    return dict_personas    

In [ ]:
def pasar_personas_a_id(dict_personas, personas):
    lista_personas = list()
    for persona in personas:
        lista_personas.append( dict_personas[persona] )
    return lista_personas    

In [ ]:
events_df.person.value_counts()

In [ ]:
dict_personas = generar_id(events_df.person)

In [ ]:
personas = pasar_personas_a_id(dict_personas, events_df.person)

In [ ]:
events_df['personas'] = personas

In [ ]:
#leo mi otro df
#nuevos_supports = pd.read_csv("/home/miki_mustard/Desktop/Facultad/Datos/TP2/nuevos_supports.csv", low_memory=False)

In [ ]:
#del(nuevos_supports)

In [ ]:
#Los uno

#events_df = events_df.join(nuevos_supports.set_index('person'), on='person', how='left')
#Memory error

In [5]:
labels_df = pd.read_csv("/home/miki_mustard/Desktop/Facultad/Datos/TP2/fiuba-trocafone-tp2-final-set/labels_training_set.csv", low_memory=False)

In [52]:
# armo df con registros completos clasificados
train_df = events_df.merge(labels_df, on='person', how='right')
train_df.shape

(1171886, 35)

In [ ]:
#train_df['new_vs_r'] = encodeado_n_vs_r

In [ ]:
#train_df = train_df.drop(columns=['new_vs_r'])

In [62]:
y_train = train_df.label

In [ ]:
y_train.shape

In [19]:
train_df = train_df.drop(columns = ['label'])

In [53]:
# armo df con registros a predecir unicamente
to_predict = events_df[~events_df.person.isin(labels_df.person)]

In [ ]:
personas = to_predict.person

In [ ]:
to_predict = to_predict.drop(columns = ['person'])

In [ ]:
#to_predict = to_predict.drop(columns = ['person'])

In [15]:
from sklearn import preprocessing
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

le = preprocessing.LabelEncoder()
le.fit(train_df.person)

LabelEncoder()

In [21]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')

enc.fit(train_df.person)
enc.fit(to_predict.person)

ValueError: Expected 2D array, got 1D array instead:
array=['ad93850f' 'ad93850f' 'ad93850f' ... '2adb3684' '39fa45b6' 'd8d6b0da'].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [16]:
le.fit(to_predict.person)

LabelEncoder()

In [23]:
train_df = train_df.drop(columns=['person'])
to_predict = to_predict.drop(columns=['person'])

In [24]:
data_dmatrix = xgb.DMatrix(data=train_df,label=y_train)

In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
    train_test_split(train_df, y_train, test_size=0.2, random_state=123)

In [26]:
xg_reg = xgb.XGBRegressor(objective ='binary:logistic',base_score= 0.3, 
                          colsample_bylevel= 0.7, booster='dart', 
                colsample_bytree = 0.4, learning_rate = 0.25,
                max_depth = 13, alpha = 25, gamma=10, n_estimators = 23, eval_metric='auc')

Exception ignored in: <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x7f0bd8e35a90>>
Traceback (most recent call last):
  File "/home/miki_mustard/envdatos/lib/python3.6/site-packages/xgboost/core.py", line 482, in __del__
    if self.handle is not None:
AttributeError: 'DMatrix' object has no attribute 'handle'


In [27]:
xg_reg.fit(X_train,y_train)

XGBRegressor(alpha=25, base_score=0.3, booster='dart', colsample_bylevel=0.7,
       colsample_bytree=0.4, eval_metric='auc', gamma=10,
       learning_rate=0.25, max_delta_step=0, max_depth=13,
       min_child_weight=1, missing=None, n_estimators=23, n_jobs=1,
       nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [28]:
preds = xg_reg.predict(X_test)

In [29]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,preds)

0.9999999930223038

In [30]:
preds_posta = xg_reg.predict(to_predict)

In [31]:
preds_posta

array([0.0038816 , 0.00341225, 0.00663358, ..., 0.00887377, 0.00948619,
       0.00823403], dtype=float32)

In [42]:
param_dist = {'objective':'binary:logistic', 'n_estimators':10, 'max_depth':10 }

In [47]:
xgclas = xgb.XGBClassifier(max_depth=3, learning_rate=0.1, n_estimators=100, silent=True, objective='binary:logistic', booster='gbtree', n_jobs=1, gamma=0, min_child_weight=1, max_delta_step=0, subsample=1, colsample_bytree=1, colsample_bylevel=1)

In [48]:
xgclas.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [ ]:
evals_result = clf.evals_result()

In [49]:
preds_xgclas = xgclas.predict_proba(to_predict)

In [51]:
preds_xgclas[:,1]

array([2.2972199e-05, 1.9855361e-04, 2.6058300e-05, ..., 1.9855361e-04,
       7.8121311e-04, 1.9855361e-04], dtype=float32)

In [55]:
import numpy as np
from catboost import CatBoostClassifier
# initialize data

In [60]:
# specify the training parameters 
model = CatBoostClassifier(iterations=5, depth=5, learning_rate=0.2)

In [63]:
#train the model
model.fit(train_df, y_train)

TypeError: Cannot convert 'b'ad93850f'' to float

In [ ]:
# make the prediction using the resulting model
preds_proba = model.predict_proba(to_predict)

print("proba = ", preds_proba)

In [ ]:
import numpy as np
from catboost import Pool, CatBoostRegressor

In [ ]:
# initialize Pool
train_pool = Pool(train_df, y_train, cat_features=[0,2,5])

In [ ]:
# specify the training parameters 
model = CatBoostRegressor(iterations=15, depth=7, learning_rate=0.3, loss_function='RMSE')

In [ ]:
#train the model
model.fit(train_pool)

In [ ]:
# make the prediction using the resulting model
preds = model.predict(to_predict)
print(preds)

In [ ]:
preds.min

In [ ]:
to_publish = pd.DataFrame()

In [ ]:
to_publish['person'] = personas

In [ ]:
to_publish['label'] = preds_posta

In [ ]:
to_publish.groupby('person', as_index=False).mean().to_csv('23_11_3.csv', index=False)

In [ ]:
to_publish = to_publish.groupby('person', as_index=False).mean()

In [ ]:
to_publish.head()

In [ ]:
to_publish.loc[to_publish.person=='00091926']

In [ ]:
to_publish.loc[to_publish.person=='00091a7a']